# tensorflow 기반 alexnet 구현

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

4422102/4422102 [==============================] - 1s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [ ]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [ ]:
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227,
                                                     input_shape=train_images.shape[1:]))

model.add(layers.Conv2D(96, (11, 11), strides=4, activation='relu')) # imagesize에 맞게 큰 filter에 큰 stride 적용
model.add(layers.Lambda(tf.nn.local_response_normalization)) # ReLU 결과값 안정화
model.add(layers.MaxPooling2D((3, 3), strides=2)) # Overlapping Pooling

model.add(layers.Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=2)) # fm = 6, 6, 256 = 9216

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000, activation='softmax'))
# model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 227, 227, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 lambda_1 (Lambda)           (None, 27, 27, 256)       0         
                                                        

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 178s 430ms/step - loss: 0.8536 - accuracy: 0.7071 - val_loss: 0.3722 - val_accuracy: 0.8600
Epoch 2/10
375/375 [==============================] - 158s 420ms/step - loss: 0.3584 - accuracy: 0.8665 - val_loss: 0.3211 - val_accuracy: 0.8777
Epoch 3/10
375/375 [==============================] - 157s 419ms/step - loss: 0.3130 - accuracy: 0.8827 - val_loss: 0.2931 - val_accuracy: 0.8876
Epoch 4/10
375/375 [==============================] - 157s 419ms/step - loss: 0.2910 - accuracy: 0.8923 - val_loss: 0.2802 - val_accuracy: 0.8972
Epoch 5/10
375/375 [==============================] - 157s 420ms/step - loss: 0.2654 - accuracy: 0.9005 - val_loss: 0.2622 - val_accuracy: 0.9065
Epoch 6/10
375/375 [==============================] - 157s 420ms/step - loss: 0.2529 - accuracy: 0.9059 - val_loss: 0.2648 - val_accuracy: 0.8994
Epoch 7/10
375/375 [==============================] - 157s 420ms/step - loss: 0.2412 - accuracy: 0.9104 - val_loss: 0.2541 -

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 5s 15ms/step - loss: 0.2764 - accuracy: 0.9050


[0.2764274775981903, 0.9049999713897705]

In [ ]:
# AlexNet 모델을 간소화해봅시다
## resize없이 (28, 28, 3)으로 받아봅시다

## ** activation 은 모두 relu로 유지
## layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기


# pytorch 기반 alexnet 구현

In [1]:
import torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4),
            nn.ReLU(),
            nn.LoacalResponseNorm(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.LocalResponseNorm(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [4]:
model = AlexNet(num_classes=1000)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=